In [7]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation,Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D

### Step 1 数据集加载及数据预处理

#### 1.1 设立超参数

In [8]:
"""
x_train,y_train：训练的样本的数据和labels 
x_test,y_test：测试的样本的数据和labels 
dtype=int,0~255 
"""

batch_size = 32 
num_classes = 10
data_augmentation = True

#### 1.2 划分训练集和测试集

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
#转成one-hot编码
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Step 2 模型搭建

#### 2.1模型介绍

Sequential:顺序模型

Activation：激活层，通过激活函数对张量进行激活 
Dense：全连接
Dropout:drouput层，以一定概率不激活神经元，防止过拟合 
Flatten：把多维输入一维化

Conv2D：2d卷积
MaxPooling2D：2d下采样,把一维的向量转换为num_class维的One-hot编码 

plot_model：打印建好的模型，相当于可视化模型

#### 2.2搭建网络

In [11]:
model = Sequential()

#第一,二层：32个（33）的卷积核，步长为1（默认也是1）
#input_shape：神经网络输入的张量的大小是多少
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
#激活函数选择‘relu’
model.add(Activation('relu'))
model.add(Dropout(0.25))
#Maxpooling:池化核的大小，22，步长strides默认和池化大小一致
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(GlobalMaxPooling2D())

# Dense（100）表示把他压成和我们labels一样的维度100，通过softmax进行激活
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_8 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 64)        0         
__________

### Step 3 训练模型
model.compile:进行编译
optimizer：随机梯度下降优化器，这里使用的是Adam，学习率是0，0001
loss='categorical_crossentropy'：多分类用的one-hot交叉熵 
model.fit(X_train,y_train,epochs=600,batch_size=128,)：进行300轮，批次为128的训练，默认训练过程中是会加入正则化防止过拟合。 loss,acc=model.evaluate(x_test,y_test)：对样本进行测试，默认不使用正则化，返回损失值和正确率。

In [12]:
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

print("train____________")
model.fit(x_train,y_train,epochs=300,batch_size=128,)
print("test_____________")

train____________
Epoch 1/300
50000/50000 [==============================] - 73s 1ms/step - loss: 2.1784 - acc: 0.1782
Epoch 2/300
50000/50000 [==============================] - 73s 1ms/step - loss: 1.9075 - acc: 0.2954
Epoch 3/300
50000/50000 [==============================] - 71s 1ms/step - loss: 1.7457 - acc: 0.3586
Epoch 4/300
50000/50000 [==============================] - 78s 2ms/step - loss: 1.6577 - acc: 0.3918
Epoch 5/300
50000/50000 [==============================] - 76s 2ms/step - loss: 1.6001 - acc: 0.4143
Epoch 6/300
50000/50000 [==============================] - 73s 1ms/step - loss: 1.5569 - acc: 0.4291
Epoch 7/300
50000/50000 [==============================] - 72s 1ms/step - loss: 1.5226 - acc: 0.4411
Epoch 8/300
50000/50000 [==============================] - 71s 1ms/step - loss: 1.4949 - acc: 0.4540
Epoch 9/300
50000/50000 [==============================] - 69s 1ms/step - loss: 1.4653 - acc: 0.4646
Epoch 10/300
50000/50000 [==============================] - 69s 1ms/step 

50000/50000 [==============================] - 68s 1ms/step - loss: 0.9113 - acc: 0.6762
Epoch 80/300
50000/50000 [==============================] - 67s 1ms/step - loss: 0.9042 - acc: 0.6817
Epoch 81/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.9005 - acc: 0.6798
Epoch 82/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.8978 - acc: 0.6833
Epoch 83/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.8927 - acc: 0.6854
Epoch 84/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.8893 - acc: 0.6849
Epoch 85/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.8839 - acc: 0.6883
Epoch 86/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.8846 - acc: 0.6868
Epoch 87/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.8828 - acc: 0.6886
Epoch 88/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.8810 - acc: 

50000/50000 [==============================] - 67s 1ms/step - loss: 0.5830 - acc: 0.7938: 0s - loss: 0.5830 - acc: 0
Epoch 235/300
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5778 - acc: 0.7950
Epoch 236/300
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5796 - acc: 0.7974
Epoch 237/300
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5761 - acc: 0.7960
Epoch 238/300
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5741 - acc: 0.7958
Epoch 239/300
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5747 - acc: 0.7972
Epoch 240/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.5724 - acc: 0.7978
Epoch 241/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.5736 - acc: 0.7969
Epoch 242/300
50000/50000 [==============================] - 68s 1ms/step - loss: 0.5708 - acc: 0.7980
Epoch 243/300
50000/50000 [==============================] 

In [13]:
loss,acc=model.evaluate(x_test,y_test)
print("loss=",loss)
print("accuracy=",acc)

10000/10000 [==============================] - 5s 453us/step
loss= 0.7131511684417725
accuracy= 0.7541
